# Assessment 1: Rule-Based AI for Credit Risk Classification

## 1. Importing Relevant Libraries    
Importing all the relevant libraries needed for this rule-based AI task.

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

## 2. Data Loading and Preparation
Here, we load the pre-split training and testing datasets. We also perform minor cleaning, such as renaming columns for easier access and removing any unnecessary index columns.

In [ ]:
try:
    train_df = pd.read_csv('credit_risk_training_data.csv')
    test_df = pd.read_csv('credit_risk_test_data.csv')
except FileNotFoundError:
    print("Error: Make sure 'credit_risk_training_data.csv' and 'credit_risk_test_data.csv' are in the correct directory.")
    exit()

for df in [train_df, test_df]:
    df.rename(columns={
        'ID': 'id',
        'Age': 'age',
        'Sex': 'sex',
        'Jobs': 'jobs',
        'Housing': 'housing',
        'Saving accounts': 'saving_accounts',
        'Checking account': 'checking_account',
        'Credit amount': 'credit_amount',
        'Duration': 'duration',
        'Purpose': 'purpose'
    }, inplace=True)
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1, inplace=True)

X_train = train_df.drop('Risk', axis=1)
y_train = train_df['Risk']
X_test = test_df.drop('Risk', axis=1)
y_test = test_df['Risk']

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

## 3. Rule-Based Classifier Class
This cell contains the core `CreditRiskClassifier` class. This class encompasses the logic for the rule-based system, including methods to add rules (the Knowledge Base) and make predictions (the Inference Engine).

In [ ]:
class CreditRiskClassifier:
    def __init__(self, default_prediction='good'):
        self.rules = []
        self.default_prediction = default_prediction

    def add_rule(self, conditions, outcome):
        self.rules.append({'conditions': conditions, 'outcome': outcome})

    def _apply_rules_to_row(self, row):
        # Applies the stored rules to a single row of data (one applicant).
        for rule in self.rules:
            match = True
            # Iterate through each condition in the rule (For example: 'age' and 'housing')
            for feature, condition in rule['conditions'].items():
                actual_value = row[feature]

                # Check if the condition is a tuple for range checking
                if isinstance(condition, tuple) and len(condition) == 2:
                    operator, value = condition
                    if operator == '>':
                        if not (actual_value > value): match = False
                    elif operator == '<':
                        if not (actual_value < value): match = False
                    elif operator == '>=':
                        if not (actual_value >= value): match = False
                    elif operator == '<=':
                        if not (actual_value <= value): match = False
                    else:
                        # Unrecognized operator
                        match = False
                
                # Fallback to the original exact match for strings or single numbers
                else:
                    if actual_value != condition:
                        match = False

                # If any condition in the rule fails, stop checking this rule
                if not match:
                    break
            
            # If all conditions in the rule passed, return the outcome
            if match:
                return rule['outcome']
        
        # If no rules matched, return the default prediction
        return self.default_prediction

    def predict(self, X):
        # Makes predictions for a whole dataset.
        return X.apply(self._apply_rules_to_row, axis=1).tolist()

## 4. Model Configuration and Rule Definition
Now we create three distinct instances of our classifier. Each instance represents one of our experimental models (Comprehensive, Financial, and Demographic). We then populate each model with its specific set of rules.

In [ ]:
# ==============================================================================
# Algorithm A: Comprehensive Baseline
# This model combines financial and demographic data for nuanced decisions.
# ==============================================================================
clf_A = CreditRiskClassifier(default_prediction='good')

# ----- High-Risk Rules (Comprehensive) -----
# Young, renting, little savings, and a large loan is a classic high-risk profile.
clf_A.add_rule({'age': ('<', 28), 'housing': 'rent', 'saving_accounts': 'little', 'credit_amount': ('>', 5000)}, 'bad')
# An unskilled worker taking a very long-term loan is risky.
clf_A.add_rule({'jobs': 0, 'duration': ('>', 36)}, 'bad')
# A very large loan for a non-essential item like furniture.
clf_A.add_rule({'purpose': 'furniture/equipment', 'credit_amount': ('>=', 7500)}, 'bad')

# ----- Low-Risk Rules (Comprehensive) -----
# The ideal applicant: older, owns a home, and has significant savings.
clf_A.add_rule({'age': ('>', 45), 'housing': 'own', 'saving_accounts': 'rich'}, 'good')
# A skilled worker with good savings taking a loan for a productive purpose is a safe bet.
clf_A.add_rule({'jobs': ('>=', 2), 'saving_accounts': 'moderate', 'purpose': 'business'}, 'good')
# Short-term loan for someone with their own home.
clf_A.add_rule({'housing': 'own', 'duration': ('<=', 12)}, 'good')


# ==============================================================================
# Algorithm B: Financial Focus
# This model makes decisions based only on the applicant's financial situation.
# ==============================================================================
clf_B = CreditRiskClassifier(default_prediction='good')

# ----- High-Risk Rules (Financial) -----
# Very little savings combined with a moderate-to-high loan amount.
clf_B.add_rule({'saving_accounts': 'little', 'credit_amount': ('>', 4000)}, 'bad')
# A large loan taken over a very long period.
clf_B.add_rule({'credit_amount': ('>', 6000), 'duration': ('>', 36)}, 'bad')
# Low savings and low checking account balance suggests poor financial health.
clf_B.add_rule({'saving_accounts': 'little', 'checking_account': 'little'}, 'bad')

# ----- Low-Risk Rules (Financial) -----
# Someone with rich savings is almost always a good risk.
clf_B.add_rule({'saving_accounts': 'rich'}, 'good')
# A small, short-term loan is very low risk.
clf_B.add_rule({'credit_amount': ('<', 1500), 'duration': ('<=', 12)}, 'good')
# Moderate savings and a reasonable checking balance is a good sign.
clf_B.add_rule({'saving_accounts': 'moderate', 'checking_account': 'moderate'}, 'good')


# ==============================================================================
# Algorithm C: Demographic Focus
# This model uses only personal information to predict risk.
# ==============================================================================
clf_C = CreditRiskClassifier(default_prediction='good')

# ----- High-Risk Rules (Demographic) -----
# A young, unskilled worker who rents their home is a high risk.
clf_C.add_rule({'age': ('<', 25), 'jobs': 0, 'housing': 'rent'}, 'bad')
# A non-essential loan for someone who is renting.
clf_C.add_rule({'housing': 'rent', 'purpose': 'furniture/equipment'}, 'bad')
# Unskilled status alone is a significant risk factor.
clf_C.add_rule({'jobs': 0}, 'bad')

# ----- Low-Risk Rules (Demographic) -----
# An older, skilled worker who owns their home represents stability.
clf_C.add_rule({'age': ('>', 40), 'jobs': ('>=', 1), 'housing': 'own'}, 'good')
# A loan for a productive purpose like 'business' or 'education' is generally safer.
clf_C.add_rule({'purpose': 'business'}, 'good')
clf_C.add_rule({'purpose': 'education'}, 'good')
# Owning a home is a strong indicator of stability.
clf_C.add_rule({'housing': 'own'}, 'good')

## 5. Evaluation Function
This helper function takes a classifier and test data, then calculates and visualizes its performance.

In [ ]:
def evaluate_model(name, classifier, X_test, y_test):
    # Calculates and prints performance metrics for a classifier.
    print(f"--- Evaluating: {name} ---")
    
    y_pred = classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, pos_label='bad', zero_division=0)
    recall = recall_score(y_test, y_pred, pos_label='bad', zero_division=0)
    f1 = f1_score(y_test, y_pred, pos_label='bad', zero_division=0)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision (for 'bad' risk): {precision:.4f}")
    print(f"Recall (for 'bad' risk): {recall:.4f}")
    print(f"F1-Score (for 'bad' risk): {f1:.4f}\n")
    
    cm = confusion_matrix(y_test, y_pred, labels=['good', 'bad'])
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Good', 'Predicted Bad'], yticklabels=['Actual Good', 'Actual Bad'])
    plt.title(f'Confusion Matrix for {name}')
    plt.show()

## 6. Run Experiment and Analyze Results
Finally, we run the evaluation for each of our three configured models on the unseen test data. The output below will form the basis of the 'Result Analysis and Discussion' section of the scientific paper.

In [ ]:
# There is a fundamental issue with one - the credit_risk_test_data.csv file does not contain a risk column
evaluate_model("Algorithm A (Comprehensive)", clf_A, X_test, y_test)
evaluate_model("Algorithm B (Financial Focus)", clf_B, X_test, y_test)
evaluate_model("Algorithm C (Demographic Focus)", clf_C, X_test, y_test)

## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## ----- Full code in one block for reference -----

In [ ]:
# import pandas as pd
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt


# # ----- 1. Data Loading and Preparation -----

# try:
#     train_df = pd.read_csv('credit_risk_training_data.csv')
#     test_df = pd.read_csv('credit_risk_test_data.csv')
# except FileNotFoundError:
#     print("Error: Make sure 'credit_risk_training_data.csv' and 'credit_risk_test_data.csv' are in the correct directory.")
#     exit()

# for df in [train_df, test_df]:
#     df.rename(columns={
#         'ID': 'id',
#         'Age': 'age',
#         'Sex': 'sex',
#         'Jobs': 'jobs',
#         'Housing': 'housing',
#         'Saving accounts': 'saving_accounts',
#         'Checking account': 'checking_account',
#         'Credit amount': 'credit_amount',
#         'Duration': 'duration',
#         'Purpose': 'purpose'
#     }, inplace=True)
#     if 'Unnamed: 0' in df.columns:
#         df.drop('Unnamed: 0', axis=1, inplace=True)

# X_train = train_df.drop('Risk', axis=1)
# y_train = train_df['Risk']
# X_test = test_df.drop('Risk', axis=1)
# y_test = test_df['Risk']

# print(f"Training set shape: {X_train.shape}")
# print(f"Testing set shape: {X_test.shape}")


# # ----- 2. Rule-Based Classifier Class -----

# class CreditRiskClassifier:
#     def __init__(self, default_prediction='good'):
#         self.rules = []
#         self.default_prediction = default_prediction

#     def add_rule(self, conditions, outcome):
#         self.rules.append({'conditions': conditions, 'outcome': outcome})

#     def _apply_rules_to_row(self, row):
#         # Applies the stored rules to a single row of data (one applicant).
#         for rule in self.rules:
#             match = True
#             # Iterate through each condition in the rule (For example: 'age' and 'housing')
#             for feature, condition in rule['conditions'].items():
#                 actual_value = row[feature]

#                 # Check if the condition is a tuple for range checking
#                 if isinstance(condition, tuple) and len(condition) == 2:
#                     operator, value = condition
#                     if operator == '>':
#                         if not (actual_value > value): match = False
#                     elif operator == '<':
#                         if not (actual_value < value): match = False
#                     elif operator == '>=':
#                         if not (actual_value >= value): match = False
#                     elif operator == '<=':
#                         if not (actual_value <= value): match = False
#                     else:
#                         # Unrecognized operator
#                         match = False
                
#                 # Fallback to the original exact match for strings or single numbers
#                 else:
#                     if actual_value != condition:
#                         match = False

#                 # If any condition in the rule fails, stop checking this rule
#                 if not match:
#                     break
            
#             # If all conditions in the rule passed, return the outcome
#             if match:
#                 return rule['outcome']
        
#         # If no rules matched, return the default prediction
#         return self.default_prediction

#     def predict(self, X):
#         # Makes predictions for a whole dataset.
#         return X.apply(self._apply_rules_to_row, axis=1).tolist()


# # ----- 3. Create And Train The Three Classifiers -----

# # ==============================================================================
# # Algorithm A: Comprehensive Baseline
# # This model combines financial and demographic data for nuanced decisions.
# # ==============================================================================
# clf_A = CreditRiskClassifier(default_prediction='good')

# # ----- High-Risk Rules (Comprehensive) -----
# # Young, renting, little savings, and a large loan is a classic high-risk profile.
# clf_A.add_rule({'age': ('<', 28), 'housing': 'rent', 'saving_accounts': 'little', 'credit_amount': ('>', 5000)}, 'bad')
# # An unskilled worker taking a very long-term loan is risky.
# clf_A.add_rule({'jobs': 0, 'duration': ('>', 36)}, 'bad')
# # A very large loan for a non-essential item like furniture.
# clf_A.add_rule({'purpose': 'furniture/equipment', 'credit_amount': ('>=', 7500)}, 'bad')

# # ----- Low-Risk Rules (Comprehensive) -----
# # The ideal applicant: older, owns a home, and has significant savings.
# clf_A.add_rule({'age': ('>', 45), 'housing': 'own', 'saving_accounts': 'rich'}, 'good')
# # A skilled worker with good savings taking a loan for a productive purpose is a safe bet.
# clf_A.add_rule({'jobs': ('>=', 2), 'saving_accounts': 'moderate', 'purpose': 'business'}, 'good')
# # Short-term loan for someone with their own home.
# clf_A.add_rule({'housing': 'own', 'duration': ('<=', 12)}, 'good')


# # ==============================================================================
# # Algorithm B: Financial Focus
# # This model makes decisions based only on the applicant's financial situation.
# # ==============================================================================
# clf_B = CreditRiskClassifier(default_prediction='good')

# # ----- High-Risk Rules (Financial) -----
# # Very little savings combined with a moderate-to-high loan amount.
# clf_B.add_rule({'saving_accounts': 'little', 'credit_amount': ('>', 4000)}, 'bad')
# # A large loan taken over a very long period.
# clf_B.add_rule({'credit_amount': ('>', 6000), 'duration': ('>', 36)}, 'bad')
# # Low savings and low checking account balance suggests poor financial health.
# clf_B.add_rule({'saving_accounts': 'little', 'checking_account': 'little'}, 'bad')

# # ----- Low-Risk Rules (Financial) -----
# # Someone with rich savings is almost always a good risk.
# clf_B.add_rule({'saving_accounts': 'rich'}, 'good')
# # A small, short-term loan is very low risk.
# clf_B.add_rule({'credit_amount': ('<', 1500), 'duration': ('<=', 12)}, 'good')
# # Moderate savings and a reasonable checking balance is a good sign.
# clf_B.add_rule({'saving_accounts': 'moderate', 'checking_account': 'moderate'}, 'good')


# # ==============================================================================
# # Algorithm C: Demographic Focus
# # This model uses only personal information to predict risk.
# # ==============================================================================
# clf_C = CreditRiskClassifier(default_prediction='good')

# # ----- High-Risk Rules (Demographic) -----
# # A young, unskilled worker who rents their home is a high risk.
# clf_C.add_rule({'age': ('<', 25), 'jobs': 0, 'housing': 'rent'}, 'bad')
# # A non-essential loan for someone who is renting.
# clf_C.add_rule({'housing': 'rent', 'purpose': 'furniture/equipment'}, 'bad')
# # Unskilled status alone is a significant risk factor.
# clf_C.add_rule({'jobs': 0}, 'bad')

# # ----- Low-Risk Rules (Demographic) -----
# # An older, skilled worker who owns their home represents stability.
# clf_C.add_rule({'age': ('>', 40), 'jobs': ('>=', 1), 'housing': 'own'}, 'good')
# # A loan for a productive purpose like 'business' or 'education' is generally safer.
# clf_C.add_rule({'purpose': 'business'}, 'good')
# clf_C.add_rule({'purpose': 'education'}, 'good')
# # Owning a home is a strong indicator of stability.
# clf_C.add_rule({'housing': 'own'}, 'good')


# # ----- 4. Evaluation -----

# def evaluate_model(name, classifier, X_test, y_test):
#     # Calculates and prints performance metrics for a classifier.
#     print(f"--- Evaluating: {name} ---")
    
#     y_pred = classifier.predict(X_test)
    
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred, pos_label='bad', zero_division=0)
#     recall = recall_score(y_test, y_pred, pos_label='bad', zero_division=0)
#     f1 = f1_score(y_test, y_pred, pos_label='bad', zero_division=0)
    
#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"Precision (for 'bad' risk): {precision:.4f}")
#     print(f"Recall (for 'bad' risk): {recall:.4f}")
#     print(f"F1-Score (for 'bad' risk): {f1:.4f}\n")
    
#     cm = confusion_matrix(y_test, y_pred, labels=['good', 'bad'])
#     plt.figure(figsize=(6, 4))
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Good', 'Predicted Bad'], yticklabels=['Actual Good', 'Actual Bad'])
#     plt.title(f'Confusion Matrix for {name}')
#     plt.show()


# # ----- 5. Run the Experiment -----
# evaluate_model("Algorithm A (Comprehensive)", clf_A, X_test, y_test)
# evaluate_model("Algorithm B (Financial Focus)", clf_B, X_test, y_test)
# evaluate_model("Algorithm C (Demographic Focus)", clf_C, X_test, y_test)

